# Advanced Lane Detection Project

## Import Packages

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip
from IPython.display import HTML

%matplotlib inline

## Classes

In [2]:
# Class to store camera calibration data
class calibration():
    def __init__(self, obj_pts, img_pts, shape):
        self.ret, self.M, self.dist, self.rvecs, self.tvecs =\
        cv2.calibrateCamera(obj_pts, img_pts, shape, None, None)

# Class to store transform matrices
class transform():
    def __init__(self, src, dst):       
        self.M = cv2.getPerspectiveTransform(src, dst)
        self.Minv = cv2.getPerspectiveTransform(dst, src)
        
# Class to store lane data
class lane():
    def __init__(self):
        self.detected = False
        self.x = None
        self.y = None
        self.fit = None

## Calibrate Camera

In [3]:
# Load calibration image filenames
fnames = glob.glob('camera_cal/calibration*.jpg')

numx = 9 # Number of inner row corners
numy = 6 # Number of inner column corners

cal_obj_pts = []
cal_img_pts = []

# Create array of known object points
obj_pts = np.zeros((numx*numy, 3), np.float32)
obj_pts[:, :2] = np.mgrid[0:numx, 0:numy].T.reshape(-1, 2)

# Loop over calibration images
for fname in fnames:
    img = mpimg.imread(fname) # Load image
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) # Convert image to grayscale
    ret, img_pts = cv2.findChessboardCorners(gray, (numx, numy), None) # Detect image points
    
    # If chessboard corners detected, append object and image points to those previously detected
    if ret == True: 
        cal_obj_pts.append(obj_pts)
        cal_img_pts.append(img_pts)

# Create an instance of the camera calibration data class
cal = calibration(cal_obj_pts, cal_img_pts, img.shape[0:2])

## Functions

In [4]:
# Thresholding function
def threshold(img, thresh):
    binary = np.zeros_like(img)
    binary[(img >= thresh[0]) & (img <= thresh[1])] = 1
    
    return binary

# Thresholded absolute Sobel gradient
def grad_thresh(img, orient, ksize, thresh):
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(img[:, :, 0], cv2.CV_64F, 1, 0, ksize))
    
    elif orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(img[:, :, 0], cv2.CV_64F, 0, 1, ksize))
    
    scaled_sobel = np.uint8(abs_sobel*(255/np.max(abs_sobel)))
    
    return threshold(scaled_sobel, thresh)

# Thresholded magnitude of Sobel gradient
def mag_thresh(img, ksize, thresh):
    sobelx = cv2.Sobel(img[:, :, 0], cv2.CV_64F, 1, 0, ksize)
    sobely = cv2.Sobel(img[:, :, 0], cv2.CV_64F, 0, 1, ksize)
    
    mag_sobelxy = np.sqrt(sobelx**2 + sobely**2)
    
    scaled_sobel = np.uint8(mag_sobelxy*(255/np.max(mag_sobelxy)))

    return threshold(scaled_sobel, thresh)

# Thresholded direction of Sobel gradient
def dir_thresh(img, ksize, thresh):
    abs_sobelx = np.absolute(cv2.Sobel(img[:, :, 0], cv2.CV_64F, 1, 0, ksize))
    abs_sobely = np.absolute(cv2.Sobel(img[:, :, 0], cv2.CV_64F, 0, 1, ksize))
    
    dir_sobelxy = np.arctan2(abs_sobely, abs_sobelx)
    
    return threshold(dir_sobelxy, thresh)

# Warp image perspective
def warp(img, view):
    # Warp image perspective into bird's eye view
    if view == 'b':
        warped = cv2.warpPerspective(img, trans.M, (img.shape[1], img.shape[0]), flags=cv2.INTER_LINEAR)
        
    # Warp image perspective into driver's view
    elif view == 'd':
        warped = cv2.warpPerspective(img, trans.Minv, (img.shape[1], img.shape[0]), flags=cv2.INTER_LINEAR)
    
    return warped

# Find initial lane locations
def find_initial(img):
    histogram = np.sum(img[img.shape[0]*(3/4):, :], axis=0) 
    midpoint = histogram.shape[0]//2
    
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint 

    left.x, left.y = find_initial_sub(img, leftx_base)
    right.x, right.y = find_initial_sub(img, rightx_base)
    
    left.fit = np.polyfit(left.y, left.x, 2)
    right.fit = np.polyfit(right.y, right.x, 2)
    
    left.detected = True
    right.detected = True
    
    return None

# Find initial lane locations - sub function
def find_initial_sub(img, xbase):
    num_windows = 9   
    window_height = img.shape[0]//num_windows # 80
    margin = 80
    min_pix = 800
    
    non_zeroy = np.array(img.nonzero()[0])
    non_zerox = np.array(img.nonzero()[1])
    
    lane_inds = []
    shift = 0
    
    # Sliding window technique
    for window in range(num_windows):
        winy_low = img.shape[0] - (window + 1)*window_height # Window lower bound
        winy_high = img.shape[0] - window*window_height # Window upper bound

        winx_left = xbase - margin # Window left bound
        winx_right = xbase + margin # Window right bound

        lane_inds_cur = ((non_zeroy >= winy_low) & (non_zeroy < winy_high) &\
                          (non_zerox >= winx_left) & (non_zerox < winx_right)).nonzero()[0]

        lane_inds.append(lane_inds_cur)
        
        # Center window on average of detected non-zero pixels
        if len(lane_inds_cur) > min_pix:
            xbase_new = np.int(np.mean(non_zerox[lane_inds_cur]))
            shift = xbase_new - xbase
            xbase = xbase_new
        
        # Shift window by previous amount
        else:
            xbase += shift
        

    lane_inds = np.concatenate(lane_inds)

    return non_zerox[lane_inds], non_zeroy[lane_inds]

# Find lane locations using margin around previous lane locations
def find_next(img):
    leftx, lefty = find_next_sub(img, left.fit)
    rightx, righty = find_next_sub(img, right.fit)
    
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Check whether current lanes are within margin of previous lanes
    if np.all([((left_fit[2] > left.fit[2]*0.8) & (left_fit[2] < left.fit[2]*1.2)),\
               ((right_fit[2] > right.fit[2]*0.8) & (right_fit[2] < right.fit[2]*1.2))]):
        
        # Compute weighted average of current and previous polynomial coefficients
        # based on number of non-zero pixels detected
        left_weights = np.array([(len(leftx)/(len(leftx) + len(left.x))),\
                                 (len(left.x)/(len(leftx) + len(left.x)))])

        right_weights = np.array([(len(rightx)/(len(rightx) + len(right.x))),\
                                 (len(right.x)/(len(rightx) + len(right.x)))])

        left.fit = np.average((left_fit, left.fit), axis = 0, weights = left_weights)
        right.fit = np.average((right_fit, right.fit), axis = 0, weights = right_weights)

        left.x, left.y = leftx, lefty
        right.x, right.y = rightx, righty
    
    # If current lanes outside of margin of previous lanes detect lanes from scratch
    else:
        find_initial(img)
    
    return None

def find_next_sub(img, fit):
    margin = 80
    
    non_zeroy = np.array(img.nonzero()[0])
    non_zerox = np.array(img.nonzero()[1])
    
    # Detect non-zero pixels within previously computed polynomial +/- margin
    lane_inds = ((non_zerox > (fit[0]*(non_zeroy**2) + fit[1]*non_zeroy + fit[2] - margin)) &\
                 (non_zerox < (fit[0]*(non_zeroy**2) + fit[1]*non_zeroy + fit[2] + margin)))
    
    return non_zerox[lane_inds], non_zeroy[lane_inds]

# Calculate road curvature radius and vehicle relation to lane center
def info(shape):
    
    # Lane base x-points
    leftx_base = np.mean(left.x[(left.y >= shape[0]*(3/4))])
    rightx_base = np.mean(right.x[(right.y >= shape[0]*(3/4))]) 
    
    # Pixel to meter conversion factors
    xconv = 3.7/(rightx_base - leftx_base)
    yconv = 30/720
    
    # Lane polynomial coefficients
    left_fit = np.polyfit(left.y*yconv, left.x*xconv, 2)
    right_fit = np.polyfit(right.y*yconv, right.x*xconv, 2)
    
    # Average of lane polynomial coefficients
    fit = np.mean((left_fit, right_fit), axis = 0)
    
    # Road curvature radius
    curve_rad = (((1 + (2*fit[0]*shape[0]*yconv + fit[1])**2)**1.5)/np.absolute(2*fit[0]))
    
    # Vehicle relation to lane center
    off_center = (shape[1]/2 - np.mean((leftx_base, rightx_base)))*xconv
    
    return curve_rad, off_center

# Draw identified lane and road information on image
def draw_lane(img):
    blank = np.zeros_like(img).astype(np.uint8)
    
    ploty = np.linspace(0, blank.shape[0] - 1, blank.shape[0])
    
    leftx = left.fit[0]*ploty**2 + left.fit[1]*ploty + left.fit[2]
    rightx = right.fit[0]*ploty**2 + right.fit[1]*ploty + right.fit[2]
    
    left_pts = np.array([np.transpose(np.vstack([leftx, ploty]))])
     # Flip right points to order polygon points from top to bottom left to bottom to top right
    right_pts = np.array([np.flipud(np.transpose(np.vstack([rightx, ploty])))])
    pts = np.hstack((left_pts, right_pts))

    return cv2.fillPoly(blank, np.int_([pts]), (0, 255, 0))

# Draw road curvature radius and vehicle relation to lane center on image
def draw_info(img):
    curve_rad, off_center = info(img.shape)
    
    curve_text = 'Curvature radius = {:d}m'.format(int(curve_rad))
    
    if off_center < 0:
        off_text = 'Vehicle {:.2f}m left of center'.format(abs(off_center))
        arrow = '>'
        arrow_pos = (1110, 620)
    
    elif off_center > 0:
        off_text = 'Vehicle {:.2f}m right of center'.format(abs(off_center))
        arrow = '<'
        arrow_pos = (100, 620)
    
    if abs(off_center) <= 0.25:
        arrow_col = (0, 255, 0)
        
    elif (abs(off_center) > 0.25) & (abs(off_center) <= 0.5):
        arrow_col = (255, 255, 0)
        
    elif abs(off_center) > 0.5:
        arrow_col = (255, 0, 0)
    
    # Draw curvature radius on image
    cv2.putText(img, curve_text, org = (10, 30), fontFace = cv2.FONT_HERSHEY_SIMPLEX,\
                fontScale = 1, thickness = 2, color = (0, 255, 0), bottomLeftOrigin = False)
    
    # Draw vehicle relation to lane center on image
    cv2.putText(img, off_text, org = (10, 60), fontFace = cv2.FONT_HERSHEY_SIMPLEX,\
                       fontScale = 1, thickness = 2, color = (0, 255, 0), bottomLeftOrigin = False)
    
    # Draw turn correction arrow on image
    return cv2.putText(img, arrow, org = arrow_pos, fontFace = cv2.FONT_HERSHEY_SIMPLEX,\
                       fontScale = 4, thickness = 4, color = arrow_col)

## Pipeline

In [5]:
def pipeline(img):
    
    # Undistort image using previously computed camera calibration matrix
    img = cv2.undistort(img, cal.M, cal.dist, None, cal.M)
    
    # Threshold image
    binary_gradx = grad_thresh(img, orient = 'x', ksize = 3, thresh = (20, 255))
    binary_grady = grad_thresh(img, orient = 'y', ksize = 3, thresh = (20, 255))
    
    binary_grad_mag = mag_thresh(img, ksize = 3, thresh = (40, 255))
    binary_grad_dir = dir_thresh(img, ksize = 3, thresh = (0.7, 1.3))
    
    binary_rch = threshold(img[:, :, 0], thresh = (140, 255))
    binary_sch = threshold(cv2.cvtColor(img, cv2.COLOR_RGB2HLS)[:, :, 2],\
                           thresh = (100, 255))
    
    # Combine binary images
    combined = np.zeros_like(binary_gradx)
    combined[((binary_gradx == 1) & (binary_grady == 1)) |\
                ((binary_grad_mag == 1) & (binary_grad_dir == 1)) |\
                ((binary_rch == 1) & (binary_sch == 1))] = 1
    
    # Warp image perspective into bird's eye view
    warped = warp(combined, view = 'b')
    
    # Find lane locations from scratch
    if left.detected == False & right.detected == False:
        find_initial(warped)
    
    # Find lane locations using margin around previously found lane locations
    elif left.detected == True & right.detected == True:
        find_next(warped)

    lane = draw_lane(img)
    unwarped = warp(lane, view = 'd')
    projected = cv2.addWeighted(img, 1, unwarped, 0.3, 0)
    
    return draw_info(projected)

## Initiate Class Instances

In [6]:
src = np.float32([[685, 450],
                  [1090, 720],
                  [190, 720],
                  [595, 450]])

dst = np.float32([[990, 0],
                  [990, 720],
                  [290, 720],
                  [290, 0]])

trans = transform(src, dst)

left = lane()
right = lane()

## Test on Video

In [7]:
video_output = 'videos/output/project_video.mp4'
video_input = VideoFileClip('videos/input/project_video.mp4')
video_clip = video_input.fl_image(pipeline)
%time video_clip.write_videofile(video_output, audio=False)

[MoviePy] >>>> Building video videos/output/project_video.mp4
[MoviePy] Writing video videos/output/project_video.mp4


100%|█████████▉| 1260/1261 [05:53<00:00,  2.86it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: videos/output/project_video.mp4 

CPU times: user 9min 4s, sys: 1min 17s, total: 10min 21s
Wall time: 5min 54s


In [8]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_output))